Доступ к АПИ

In [321]:
import requests

In [322]:
API_KEY='J2CB0NW-FWXMZ4E-GSHP4Y9-M32XB7Z'

Список фильмов, с которыми будем работать + доп. информация

In [323]:
movies = {
    '1+1': 535341,
    'Побег из Шоушенка': 326,
    'Зеленая миля': 435,
    'Остров проклятых': 397667,
    'Бойцовский клуб': 361,
    'Форрест Гамп': 448,
    'Унесенные призраками': 370,
    'Джентльмены': 1143242,
    'Леон': 389
}

headers = {
    'X-API-KEY': API_KEY
}

In [324]:
def get_reviews(movies, url, headers):
    top_movies = []
    for movie_name in movies:
        new_url = f'{url}{movies[movie_name]}'
        response = requests.get(new_url, headers=headers)
        if response.status_code == 200:
            top_movies.append(response.json())
        else:
            print(f'Error: {response.status_code} - {response.text}')
    return top_movies

Получение отзывов

In [325]:
URL_NEGATIVE='https://api.kinopoisk.dev/v1.4/review?page=1&limit=15&selectFields=review&type=Негативный&movieId='
URL_POSITIVE='https://api.kinopoisk.dev/v1.4/review?page=1&limit=20&selectFields=review&type=Позитивный&movieId='

In [326]:
negative_reviews = get_reviews(movies, URL_NEGATIVE, headers)
negative_reviews

[{'docs': [{'review': '1+1 — это один из тех фильмов, которые все видели, но не многие смотрели. Я, как и многие, представлял себе, что этот фильм — история о столкновении двух миров: богатства и бедности, человека, что полностью здоров, и того, кто прикован к инвалидному креслу.\r\n\r\nИ на бумаге это смотрится свежо и интересно, герои дополняют друг друга, параллельно учась жить с тем, чем их наградила судьба (если вы, конечно, верите в концепцию судьбы). Но, к большому для меня сожалению, всё оказалось далеко не так, как я думал.\r\n\r\nЕсть 2 главных героя: Филипп, что является богатым аристократом, и Дрисс, что является бедняком из бедного района Парижа. Изначально лишь Дрисса преподносят как человека не самого хорошего, он обманывает, ворует и ведёт асоциальный образ жизни. Филипп же на его фоне выглядит образцом адекватности, он начитан, терпелив и прямолинейно честен, но с появлением в жизни Дрисса его характер меняется в худшую сторону.\r\n\r\nА к концу фильма уже оба персонаж

In [327]:
positive_reviews = get_reviews(movies, URL_POSITIVE, headers)
positive_reviews

[{'docs': [{'review': "Говорить что-то сейчас о '1+1' практически не имеет смысла. Фильм стал чуть ли не культовым, и практически нет любителей большого кино, которые бы его не видели или хотя бы не слышали.\r\n\r\nРассказываемая история на самом деле даже не столько драма, сколько повесть, в чем-то жизнеутверждающая. Дружба двух людей из совершенно разных социальных слоев, разных настолько, что их можно считать буквально разными мирами. Дружба людей, которые в иное время не могли пересечься нигде, но трагедия свела-таки их вместе.\r\n\r\n'1+1' это приятный фильм, качественно снятый, хорошо написанный и талантливо сыгранный. Главный экранный дуэт, Франсуа Клюзе и Омар Си, очень хорошо гармонируют вместе, пусть поначалу их образы и смотрятся максимально контрастно. Однако химия между героями определенно чувствуется, и на ней строится львиная доля очарования ленты.\r\n\r\nПравда, есть все же в ленте даже не недостаток, а условие, которая делает фильм достаточно легким для просмотра, несм

In [328]:
from bs4 import BeautifulSoup

In [329]:
def remove_html_tags(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text()

In [330]:
def get_all_reviews(reviews):
    reviews_get = []
    for movie in reviews:
        for review in movie['docs']:
            proc_review = '\n'.join(line for line in review['review'].split('\n') if line.strip())
            proc_review = remove_html_tags(proc_review)
            reviews_get.append(proc_review)
    return reviews_get

In [331]:
positive_reviews_get = get_all_reviews(positive_reviews)[:130]
negative_reviews_get = get_all_reviews(negative_reviews)

In [332]:
print(len(positive_reviews_get))
print(len(negative_reviews_get))

130
125


In [333]:
import pandas as pd

In [334]:
df = pd.DataFrame({
    'Review': positive_reviews_get + negative_reviews_get,
    'Tone': [1 for i in range(130)] + [0 for i in range(125)]
})
df = df.sample(frac=1).reset_index(drop=True)
df.head(15)

,Review,Tone
0,"Гениальное произведение Чака Паланика, снятое ...",1
1,'Побег из Шоушенка' стал моим знакомством с тв...,0
2,"Этот фильм я презираю, и это не просто эмоцион...",0
3,"Мне через 2 месяца будет 30 лет, и только вчер...",1
4,С трудом удалось досмотреть сиё творение. Собс...,0
5,Фильм рассказывает нам о двух уникальных жизне...,1
6,"Знаете, вот сейчас сажусь я писать рецензию на...",1
7,"Довольно неоднозначный фильм. С одной стороны,...",0
8,В России здорово переоценивают этого режиссера...,0
9,Смотрела я этот мультфильм довольно давно и оч...,0


In [335]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [336]:
x = df['Review']
y = df['Tone']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

model = MultinomialNB()
model.fit(x_train_tfidf, y_train)

MultinomialNB()

In [337]:
y_pred = model.predict(x_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.90


In [338]:
URL_NEGATIVE_2='https://api.kinopoisk.dev/v1.4/review?page=2&limit=10&selectFields=review&type=Негативный&movieId='
URL_POSITIVE_2='https://api.kinopoisk.dev/v1.4/review?page=2&limit=10&selectFields=review&type=Позитивный&movieId='

In [339]:
positive_reviews_2 = get_reviews(movies, URL_POSITIVE_2, headers)
positive_reviews_2

[{'docs': [{'review': '1+1 - французский фильм о невероятной дружбе между богатым парализованным инвалидом и его опекуном, молодым человеком из гетто. Фильм вышел на экраны в 2011 году и имел огромный кассовый успех во Франции, побив несколько рекордов и став самым кассовым французским фильмом в истории.\r\n\r\nВ целом, я бы поставил фильму оценку 7 из 10. В фильме содержится трогательное и воодушевляющее послание о силе дружбы и о том, как важно не судить людей по их внешности или обстоятельствам. Выступления Франсуа Клюзе в роли парализованного Филиппа и Омара Си в роли его смотрителя Дрисса превосходны, и их взаимная химия - один из самых ярких моментов фильма.\r\n\r\nОдна из вещей, которая мне больше всего понравилась в 1+1, - это то, как в ней тонко и вдумчиво рассматриваются темы привилегий и неравенства. Несмотря на различия в их происхождении, Филипп и Дрисс образуют прочную связь и учатся друг у друга способами, выходящими за рамки их профессиональных отношений.\r\n\r\nОднако 

In [340]:
negative_reviews_2 = get_reviews(movies, URL_NEGATIVE_2, headers)
negative_reviews_2

[{'docs': [{'review': 'Ну и вот, открываю это я список французских фильмов снятых за всю историю синематографа (изобретенного французами, кстати). В списке значится 34 140 фильмов пока. На первом месте находится сабж. Что ж, и не зря. Ведь в этом фильме все прекрасно: и безработный вор из Сенегала, и парализованный бездетный аристократ, и драгоценные живописные кляксы, и секретарша-лесбиянка (ну слава богу, а то мы уж распереживались, чо это она нег... афросенегальцу не дает. Уж не расистка ли? А, нет, все в порядке, причина более чем уважительная), и недоделанные, нудные, беспомощные бледнолицые французишки, которых ставит на место африканский герой, раздавая воспитательные затрещины. Все повествование проникнуто искрометным политкорректным юмором, дабы какой зритель не возомнил о себе ненароком чего лишнего - образования там какого... среднего.. или возраста старше 10 лет - все шутки будут изысканно незамысловаты: нет ручек - нет конфетки (ггг), показал свой огурчик (ггг), а давайте 

In [341]:
for movie in movies:
    movies[movie] = {'positive': [], 'negative': [], 'reviews': []}

In [342]:
for movie1, movie2, movie in zip(positive_reviews_2, negative_reviews_2, movies):
    for review1, review2 in zip(movie1['docs'], movie2['docs']):
        proc_review_1 = '\n'.join(line for line in review1['review'].split('\n') if line.strip())
        proc_review_1 = remove_html_tags(proc_review_1)
        proc_review_2 = '\n'.join(line for line in review2['review'].split('\n') if line.strip())
        proc_review_2 = remove_html_tags(proc_review_2)
        movies[movie]['reviews'].append(proc_review_1)
        movies[movie]['reviews'].append(proc_review_2)

In [343]:
movies

{'1+1': {'positive': [],
  'negative': [],
  'reviews': ['1+1 - французский фильм о невероятной дружбе между богатым парализованным инвалидом и его опекуном, молодым человеком из гетто. Фильм вышел на экраны в 2011 году и имел огромный кассовый успех во Франции, побив несколько рекордов и став самым кассовым французским фильмом в истории.\r\nВ целом, я бы поставил фильму оценку 7 из 10. В фильме содержится трогательное и воодушевляющее послание о силе дружбы и о том, как важно не судить людей по их внешности или обстоятельствам. Выступления Франсуа Клюзе в роли парализованного Филиппа и Омара Си в роли его смотрителя Дрисса превосходны, и их взаимная химия - один из самых ярких моментов фильма.\r\nОдна из вещей, которая мне больше всего понравилась в 1+1, - это то, как в ней тонко и вдумчиво рассматриваются темы привилегий и неравенства. Несмотря на различия в их происхождении, Филипп и Дрисс образуют прочную связь и учатся друг у друга способами, выходящими за рамки их профессиональны

In [344]:
for movie in movies:
    if len(movies[movie]['reviews']) == 0:
        movies[movie] = []
    else:
        tmp_df = pd.DataFrame({
            'Review': movies[movie]['reviews']
        })
        x_test_tfidf = vectorizer.transform(tmp_df['Review'])
        y_pred = model.predict(x_test_tfidf)
        for i in range(len(y_pred)):
            if y_pred[i] == 1:
                movies[movie]['positive'].append(movies[movie]['reviews'][i])
            else:
                movies[movie]['negative'].append(movies[movie]['reviews'][i])

In [345]:
for movie in movies:
    if movies[movie] != []:
        movies[movie]['positive'] = " ".join(movies[movie]['positive'])
        movies[movie]['negative'] = " ".join(movies[movie]['negative'])


In [350]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
nltk.download('punkt_tab')

def summarize_text(text, max_sentences=5):
    # Загружаем стоп-слова
    stop_words = set(stopwords.words("russian"))
    
    # Разбиваем текст на предложения
    sentences = sent_tokenize(text)
    
    # Составляем список слов, учитывая их частоту встречаемости
    freq_table = dict()
    for sent in sentences:
        words = word_tokenize(sent)
        for word in words:
            if word not in stop_words:
                if word in freq_table:
                    freq_table[word] += 1
                else:
                    freq_table[word] = 1
    
    # Вычисляем вес каждого предложения
    sentence_weights = dict()
    for sent in sentences:
        for word, freq in freq_table.items():
            if word in sent.lower():
                if sent in sentence_weights:
                    sentence_weights[sent] += freq
                else:
                    sentence_weights[sent] = freq

    # Вычисляем средний вес предложения
    avg_weight = sum(sentence_weights.values()) / len(sentence_weights)
    
    # Формируем итоговую суммаризацию
    summary = []
    for sent in sentences:
        if sent in sentence_weights and sentence_weights[sent] > (1.2 * avg_weight):
            summary.append(sent)
    
    # Ограничиваем длину суммаризации по количеству предложений
    summary = summary[:max_sentences]
    return ' '.join(summary)

[nltk_data] Downloading package stopwords to /Users/dara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/dara/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [351]:
data = {}
for movie in movies:
    if movies[movie] != []:
        data[movie] = {
            'positive': summarize_text(movies[movie]['positive']),
            'negative': summarize_text(movies[movie]['negative'])
        }

In [352]:
import json

In [353]:
with open('movies.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)